In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import * 
from pyspark.storagelevel import StorageLevel
from pyspark.sql import Row

In [2]:
import pandas as pd
# warehouse
dfw = pd.DataFrame({'code': [1, 2, 3, 4, 5],
          'location': ['Chicago', 'Chicago', 'New York', 'Los Angeles', 'San Francisco'],
          'capacity': [3, 4, 7, 2, 8]})

schema = StructType([
    StructField('code',IntegerType(),True),
    StructField('location',StringType(),True),
    StructField('capacity',IntegerType(),True),
    ])

sdfw = sqlContext.createDataFrame(dfw, schema)
sdfw.createOrReplaceTempView("Warehouses")


# box
dfb = pd.DataFrame({'code': ['0MN7', '4H8P', '4RT3', '7G3H', '8JN6', '8Y6U', '9J6F', 'LL08', 'P0H6', 'P2T6', 'TU55'],
          'contents': ['Rocks', 'Rocks', 'Scissors', 'Rocks', 'Papers', 'Papers', 'Papers', 'Rocks', 'Scissors', 'Scissors', 'Papers'],
          'value': [180.0, 250.0, 190.0, 200.0, 75.0, 50.0, 175.0, 140.0, 125.0, 150.0, 90.0],
          'warehouse': [3, 1, 4, 1, 1, 3, 2, 4, 1, 2, 5]})

schema = StructType([
    StructField('code',StringType(),True),
    StructField('contents',StringType(),True),
    StructField('value',FloatType(),True),
    StructField('warehouse',IntegerType(),True),

    ])
sdfw.show()
sdfb.show()
sdfb = sqlContext.createDataFrame(dfb, schema)
sdfb.createOrReplaceTempView("Boxes")


sdfb.groupBy("contents").agg(F.count("contents").alias("count_boxes")).show()

+----+-------------+--------+
code| location|capacity|
+----+-------------+--------+
 1| Chicago| 3|
 2| Chicago| 4|
 3| New York| 7|
 4| Los Angeles| 2|
 5|San Francisco| 8|
+----+-------------+--------+

+----+--------+-----+---------+
code|contents|value|warehouse|
+----+--------+-----+---------+
0MN7| Rocks|180.0| 3|
4H8P| Rocks|250.0| 1|
4RT3|Scissors|190.0| 4|
7G3H| Rocks|200.0| 1|
8JN6| Papers| 75.0| 1|
8Y6U| Papers| 50.0| 3|
9J6F| Papers|175.0| 2|
LL08| Rocks|140.0| 4|
P0H6|Scissors|125.0| 1|
P2T6|Scissors|150.0| 2|
TU55| Papers| 90.0| 5|
+----+--------+-----+---------+

+--------+---------------+
contents|count(contents)|
+--------+---------------+
 Rocks| 4|
 Papers| 4|
Scissors| 3|
+--------+---------------+

In [3]:
sdfw=sdfw.withColumnRenamed("code","code1")
sdfw.join(sdfb, sdfw.code1==sdfb.warehouse).groupBy("code1").agg(F.count("code").alias("count"),F.first("capacity").alias("capacity")).filter("count>capacity").select(F.col("code1").alias("code")).show()

+----+
code|
+----+
 1|
+----+

In [4]:
sdfw.join(sdfb, sdfw.code1==sdfb.warehouse).show()

+-----+-------------+--------+----+--------+-----+---------+
code1| location|capacity|code|contents|value|warehouse|
+-----+-------------+--------+----+--------+-----+---------+
 1| Chicago| 3|4H8P| Rocks|250.0| 1|
 1| Chicago| 3|7G3H| Rocks|200.0| 1|
 1| Chicago| 3|8JN6| Papers| 75.0| 1|
 1| Chicago| 3|P0H6|Scissors|125.0| 1|
 3| New York| 7|0MN7| Rocks|180.0| 3|
 3| New York| 7|8Y6U| Papers| 50.0| 3|
 5|San Francisco| 8|TU55| Papers| 90.0| 5|
 4| Los Angeles| 2|4RT3|Scissors|190.0| 4|
 4| Los Angeles| 2|LL08| Rocks|140.0| 4|
 2| Chicago| 4|9J6F| Papers|175.0| 2|
 2| Chicago| 4|P2T6|Scissors|150.0| 2|
+-----+-------------+--------+----+--------+-----+---------+

In [5]:
spark.sql("""
select w.code
from Warehouses w
join Boxes b
on w.code = b.warehouse
group by w.code
having count(b.code) > first(w.capacity)

""").show()

+----+
code|
+----+
 1|
+----+

In [6]:
dict ={11: {101, 102}, 22: {111, 112, 101}, 33: {101}, 44: {102}}

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2997264772801405> in <module> 
 ----> 1 { [ str ( x ) , list ( y ) ] for x , y in dict . items ( ) } 

 <command-2997264772801405> in <setcomp> (.0) 
 ----> 1 { [ str ( x ) , list ( y ) ] for x , y in dict . items ( ) } 

 TypeError : unhashable type: 'list'

In [8]:
dict ={11: {101, 102}, 22: {111, 112, 101}, 33: {101}, 44: {102}}

In [9]:
dict ="{11: {101, 102}, 22: {111, 112, 101}, 33: {101}, 44: {102}}"

In [10]:
dict ="{'11': [101, 102], '22': [111, 112, 101], '33': [101], '44': [102]}"

In [11]:
df=spark.createDataFrame([[dict]],['col1'])
df.show(truncate=False)
a=df.withColumn("col1", F.regexp_replace(F.regexp_replace(F.regexp_replace("col1", "\{","\["),"\}","\]"),"\:",",")).collect()

+-----------------------------------------------------------+
col1 |
+-----------------------------------------------------------+
{11: {101, 102}, 22: {111, 112, 101}, 33: {101}, 44: {102}}|
+-----------------------------------------------------------+

In [13]:
schema=MapType(StringType(),ArrayType(IntegerType()))

df1.withColumn("col1", F.from_json("col1", schema)).select(F.explode("col1").alias("x","y")).select("x",F.explode("y").alias("y")).show()


+---+---+
 x| y|
+---+---+
+---+---+

In [14]:
yo=[]
for key, value in dict.items():
    yo.append([key,list(value)])
    

In [15]:
from pyspark.sql import functions as F
yo=[]
for key, value in dict.items():
    yo.append([key,list(value)])
df=spark.createDataFrame(yo,['user_id','team_id'])
df.select("user_id", F.explode("team_id")).show()
yo

+-------+---+
user_id|col|
+-------+---+
 11|101|
 11|102|
 22|112|
 22|101|
 22|111|
 33|101|
 44|102|
+-------+---+

Out[18]: [[11, [101, 102]], [22, [112, 101, 111]], [33, [101]], [44, [102]]]

In [16]:
[(x,list(y)) for x,y in dict.items()]*5000

Out[32]: [(11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102]),
 (22, [112, 101, 111]),
 (33, [101]),
 (44, [102]),
 (11, [101, 102

In [17]:
df=spark.createDataFrame([[x,list(y)] for x,y in dict.items()]*50000,['user_id','team_id'])
df.select("user_id", F.explode("team_id").alias("team_id")).show()

+-------+-------+
user_id|team_id|
+-------+-------+
 11| 101|
 11| 102|
 22| 112|
 22| 101|
 22| 111|
 33| 101|
 44| 102|
 11| 101|
 11| 102|
 22| 112|
 22| 101|
 22| 111|
 33| 101|
 44| 102|
 11| 101|
 11| 102|
 22| 112|
 22| 101|
 22| 111|
 33| 101|
+-------+-------+
only showing top 20 rows

In [18]:
df=spark.createDataFrame(yo,['col1','col2'])

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/types.py in _infer_type (obj) 
 1035 try : 
 -> 1036 return _infer_schema ( obj ) 
 1037 except TypeError : 

 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1061 else : 
 -> 1062 raise TypeError ( "Can not infer schema for type: %s" % type ( row ) ) 
 1063 

 TypeError : Can not infer schema for type: <class 'set'>

During handling of the above exception, another exception occurred:

 TypeError Traceback (most recent call last)
 <command-3407523990111840> in <module> 
 ----> 1 df = spark . createDataFrame ( yo , [ 'col1' , 'col2' ] ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 815 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 816 else : 
 --> 817 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 818 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 819 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 440 write temp files . 
 441 """
 --> 442 data , schema = self . _wrap_data_schema ( data , schema ) 
 443 return self . _sc . parallelize ( data ) , schema
 444 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 419 
 420 if schema is None or isinstance ( schema , ( list , tuple ) ) : 
 --> 421 struct = self . _inferSchemaFromList ( data , names = schema ) 
 422 converter = _create_converter ( struct ) 
 423 data = map ( converter , data ) 

 /databricks/spark/python/pyspark/sql/session.py in _inferSchemaFromList (self, data, names) 
 355 warnings.warn("inferring schema from dict is deprecated,"
 356 "please use pyspark.sql.Row instead")
 --> 357 schema = reduce ( _merge_type , ( _infer_schema ( row , names ) for row in data ) ) 
 358 if _has_nulltype ( schema ) : 
 359 raise ValueError ( "Some of types cannot be determined after inferring" ) 

 /databricks/spark/python/pyspark/sql/session.py in <genexpr> (.0) 
 355 warnings.warn("inferring schema from dict is deprecated,"
 356 "please use pyspark.sql.Row instead")
 --> 357 schema = reduce ( _merge_type , ( _infer_schema ( row , names ) for row in data ) ) 
 358 if _has_nulltype ( schema ) : 
 359 raise ValueError ( "Some of types cannot be determined after inferring" ) 

 /databricks/spark/python/pyspark/sql/types.py in _infer_schema (row, names) 
 1062 raise TypeError ( "Can not infer schema for type: %s" % type ( row ) ) 
 1063 
 -> 1064 fields = [ StructField ( k , _infer_type ( v ) , True ) for k , v in items ] 
 1065 return StructType ( fields ) 
 1066 

 /databricks/spark/python/pyspark/sql/types.py in <listcomp> (.0) 
 1062 raise TypeError ( "Can not infer schema for type: %s" % type ( row ) ) 
 1063 
 -> 1064 fields = [ StructField ( k , _infer_type ( v ) , True ) for k , v in items ] 
 1065 return StructType ( fields ) 
 1066 

 /databricks/spark/python/pyspark/sql/types.py in _infer_type (obj) 
 1036 return _infer_schema ( obj ) 
 1037 except TypeError : 
 -> 1038 raise TypeError ( "not supported type: %s" % type ( obj ) ) 
 1039 
 1040 

 TypeError : not supported type: <class 'set'>

In [19]:
from pyspark.sql import functions as F

In [20]:
list1=  [['Cat'   ,3],   
        ['Dog'   ,2],   
        ['Goose' ,1]]
list2= [[ 1,'CAT',10.0],
        [ 1,'DOG',1.0],
        [ 1,'GOOSE',0.1],
        [ 2,'CAT',0.01],
        [ 2,'DOG',0.001],
        [ 3,'GOOSE',0.0001]]

df=spark.createDataFrame(list1,['Type','Value'])
df1=spark.createDataFrame(list2,['ID','ITEM','QUANTITY'])

df.show()
df1.show()


df1.join(df,F.expr("""ITEM=upper(Type)""")).drop("Type")\
   .withColumn("Value", F.col("Value")*F.col("QUANTITY")).orderBy("ID","ITEM").show(truncate=False)
        

+-----+-----+
 Type|Value|
+-----+-----+
 Cat| 3|
 Dog| 2|
Goose| 1|
+-----+-----+

+---+-----+--------+
 ID| ITEM|QUANTITY|
+---+-----+--------+
 1| CAT| 10.0|
 1| DOG| 1.0|
 1|GOOSE| 0.1|
 2| CAT| 0.01|
 2| DOG| 0.001|
 3|GOOSE| 1.0E-4|
+---+-----+--------+

+---+-----+--------+------+
ID |ITEM |QUANTITY|Value |
+---+-----+--------+------+
1 |CAT |10.0 |30.0 |
1 |DOG |1.0 |2.0 |
1 |GOOSE|0.1 |0.1 |
2 |CAT |0.01 |0.03 |
2 |DOG |0.001 |0.002 |
3 |GOOSE|1.0E-4 |1.0E-4|
+---+-----+--------+------+

In [21]:
.drop("Type")\
   .withColumn("Value", F.col("Value")*F.col("QUANTITY")).show()

In [22]:
+----+-------+----------+--------+
| ID | ITEM  | QUANTITY | Value  |
+====+=======+==========+========+
| 1  | CAT   | 10.0     | 30     |
+----+-------+----------+--------+
| 1  | DOG   | 1.0      | 2      |
+----+-------+----------+--------+
| 1  | GOOSE | 0.1      | 0.1    |
+----+-------+----------+--------+
| 2  | CAT   | 0.01     | 0.03   |
+----+-------+----------+--------+
| 2  | DOG   | 0.001    | 0.002  |
+----+-------+----------+--------+
| 3  | GOOSE | 0.0001   | 0.0001 |
+----+-------+----------+--------+

In [23]:
 1  | CAT   | 10.0     |
+----+-------+----------+
| 1  | DOG   | 1.0      |
+----+-------+----------+
| 1  | GOOSE | 0.1      |
+----+-------+----------+
| 2  | CAT   | 0.01     |
+----+-------+----------+
| 2  | DOG   | 0.001    |
+----+-------+----------+
| 3  | GOOSE | 0.0001  